In [14]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [4]:
# Define the language code, used in the file names
lang_code = "CZ"

# Main path
main_path = "/home/tajak/Parlamint-translation"

# --------------------NO CHANGING OF THE CODE NEEDED FROM NOW ONWARDS------------------
import pandas as pd

translated_dataframe_path = "{}/results/{}/ParlaMint-{}-translated.csv".format(main_path, lang_code, lang_code)
translated_tokenized_dataframe_path = "{}/results/{}/ParlaMint-{}-translated-tokenized.csv".format(main_path,lang_code, lang_code)
final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

# Debugging aligning

In [8]:
df.iloc[1236912].to_dict()

{'file_path': '/home/tajak/Parlamint-translation/Source-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2015/ParlaMint-CZ_2015-01-23-ps2013-025-04-004-088.conllu',
 'file': 'ParlaMint-CZ_2015-01-23-ps2013-025-04-004-088.conllu',
 'sentence_id': 'ParlaMint-CZ_2015-01-23-ps2013-025-04-004-088.u8.p1.s5',
 'text': 'To -',
 'tokenized_text': 'To -',
 'proper_nouns': '{}',
 'length': 2,
 'translation': "It's--",
 'translation-tokenized': "It's--",
 'space-after-information': "['Last']"}

In [26]:
aligns_list = open("eflomal/source-en_{}.rev".format(lang_code), "r").readlines()

aligns_list = aligns_list[:10]

# Create forward target alignments from the source alignment direction (by changing the direction in the rev file)
#aligns_list = open("source-en_{}.rev".format(lang_code), "r").readlines()
aligns_list = [i.replace("\n", "") for i in aligns_list]

# Continue with processing the list to create the final alignments format which I'll use to correct proper names
aligns_list = [i.split(" ") for i in aligns_list]

for i in aligns_list:
	# If alignment line is empty, keep the dictionary empty
	if len(i) == 1 and len(i[0]) == 0:
		aligns_list[aligns_list.index(i)] = []
	else:
		for pair in i:
			current_pair = pair.split("-")
			i[i.index(pair)] = {int(current_pair[0]): int(current_pair[1])}

len(aligns_list)

10

In [27]:
final_aligns = []

# Create a dictionary out of the rev alignments
for i in aligns_list:
	current_line = {}

	try:
		for element in i:
			a = list(element.items())[0][0]
			b = list(element.items())[0][1]
			current_line[a] = b
	
		# Check whether the number of pairs in the list is the same as number of items
		if len(i) != len(list(current_line.items())):
			print("Not okay:")
			print(i)
			print(current_line)

		final_aligns.append(current_line)
	
	except:
		print("error")
		print(aligns_list.index(i))
		print(i)
		final_aligns.append("Error")
	
print("\nNumber of aligned sentences: {}\n\n".format(len(final_aligns)))

len(final_aligns)


Number of aligned sentences: 10




10

In [28]:
# Create a dictionary from the returned alignment files which will be added to each word in the final conllu

def alignment_file_to_target_dict(file):
	"""
	The output of the eflomal aligner is in the source to target direction. We want to get the alignments in the other direction
	and for each target word add to the conllu its aligned source word index (as it appears in conllu). In conllu, indices start
	with 1, not 0. So, we take the eflomal files, reverse the order and create dictionaries with target indexes as keys
	and source indexes as values. If there are more than one words aligned to the same target word, it looks like this: '1, 2'.
	We use the conllu indexes which means that we add 1 to each index in the alingment pairs. 

	Args:
		- file: the path to the .fwd and .rev file that is produced by the eflomal tool

	The result is a list of dictionaries, each dictionary corresponds to one sentence.
	"""
	# Create target alignments from the source alignment direction (by changing the direction in the file)
	aligns_list_target = open(file, "r").readlines()
	aligns_list_target = [i.replace("\n", "") for i in aligns_list_target]
	aligns_list_target = [i.split(" ") for i in aligns_list_target]

	aligns_list_target_dict_list = []

	# Loop through the alignments for sentences
	for i in aligns_list_target:
		# Create a dictionary for each sentence
		current_sentence_align = {}
		# If alignment line is empty, keep the dictionary empty
		if len(i) == 1 and len(i[0]) == 0:
			current_sentence_align = {}
		else:
			# For each alignment pair in the sentence:
			for pair in i:
				# Split the pair: result is a list of lists with source index as the first element
				# and target index as the second element: [[0,0], [1,2], [1,3]]
				current_pair = pair.split("-")

				# Get the indices for target and source and add 1 to them (to get the conllu indices)
				current_t_index = int(current_pair[1]) + 1
				current_s_index = int(current_pair[0]) + 1

				# Check whether the target index is already aligned to anything (a case of 1-to-many alignment),
				# if not, save it as a key and save the source index as value.
				if current_sentence_align.get(current_t_index, None) == None:
					current_sentence_align[current_t_index] = str(current_s_index)
				# If the index was already aligned to a previous source word, add the additional source word alignment as a string
				# (result: {0: "1, 2"))
				else:
					current_sentence_align[current_t_index] += str(", ")
					current_sentence_align[current_t_index] += str(current_s_index)

		aligns_list_target_dict_list.append(current_sentence_align)

	return aligns_list_target_dict_list

def correct_proper_nouns(translated_tokenized_dataframe_path, final_dataframe, lang_code):
	"""
	This function takes the translated text and the source text, aligns words with eflomal and corrects proper nouns.
	It takes the dataframe that was created in the function extract_text() and to which the translation was added
	in the function translate().

	To use eflomal, you need to install it first:
	!git clone https://github.com/robertostling/eflomal
	%cd eflomal
	!make
	!sudo make install
	!python3 setup.py install

	In case you don't have sudo permission, you can skip !sudo make install. I did, and I also used a virtual environment (venv), and managed to install eflomal.

	"""
	import pandas as pd
	import re
	import ast
	from IPython.display import display
	import time
	import subprocess
	import os

	# Open the file, created in the previous step
	df = pd.read_csv("{}".format(translated_tokenized_dataframe_path), sep="\t", index_col=0)

	# Move into the eflomal folder
	os.chdir("/home/tajak/Parlamint-translation/eflomal")

	# Then we need to create files for all texts and all translations
	source_sentences = open("source_sentences_{}.txt".format(lang_code), "w")
	English_sentences = open("English_sentences_{}.txt".format(lang_code), "w")

	for i in df["tokenized_text"].to_list():
		source_sentences.write(i)
		source_sentences.write("\n")

	for i in df["translation-tokenized"].to_list():
		English_sentences.write(i)
		English_sentences.write("\n")

	source_sentences.close()
	English_sentences.close()

	print("\n\n")
	print("Alignment started.")
	start_time = time.time()

	# Align sentences with eflomal and get out a file with alignments
	#!python3 align.py -s source_sentences.txt -t English_sentences.txt --model 3 -r source-en.rev -f source-en.fwd
	subprocess.call(["/home/tajak/Parlamint-translation/align.sh", lang_code])

	# Create a list of dictionaries of alignments from the returned files which will be added to the final conllu for each word
	forward_alignment_dict_list = alignment_file_to_target_dict("source-en_{}.fwd".format(lang_code))
	backward_alignment_dict_list = alignment_file_to_target_dict("source-en_{}.rev".format(lang_code))

	# Add to the df
	df["fwd_align_dict"] = forward_alignment_dict_list
	df["bwd_align_dict"] = backward_alignment_dict_list

	# Create forward target alignments from the source alignment direction (by changing the direction in the rev file)
	aligns_list = open("source-en_{}.rev".format(lang_code), "r").readlines()
	aligns_list = [i.replace("\n", "") for i in aligns_list]

	# Continue with processing the list to create the final alignments format which I'll use to correct proper names
	aligns_list = [i.split(" ") for i in aligns_list]

	for i in aligns_list:
		# If alignment line is empty, keep the dictionary empty
		if len(i) == 1 and len(i[0]) == 0:
			aligns_list[aligns_list.index(i)] = []
		else:
			for pair in i:
				current_pair = pair.split("-")
				i[i.index(pair)] = {int(current_pair[0]): int(current_pair[1])}
	
	final_aligns = []

	# Create a dictionary out of the rev alignments
	for i in aligns_list:
		current_line = {}

		try:
			for element in i:
				a = list(element.items())[0][0]
				b = list(element.items())[0][1]
				current_line[a] = b
		
			# Check whether the number of pairs in the list is the same as number of items
			if len(i) != len(list(current_line.items())):
				print("Not okay:")
				print(i)
				print(current_line)

			final_aligns.append(current_line)
		
		except:
			print("error")
			print(aligns_list.index(i))
			print(i)
			final_aligns.append("Error")
		
	print("\nNumber of aligned sentences: {}\n\n".format(len(final_aligns)))

	# Add a to the df
	df["alignments"] = final_aligns

	# When we open the dataframe file, the dictionaries with proper names changed into strings - Change strings in the column proper_nouns into dictionaries

	df["proper_nouns"] = df.proper_nouns.astype("str")
	df["proper_nouns"] = df.proper_nouns.apply(lambda x: ast.literal_eval(x))

	# Change nan values in the proper_nouns columns
	df = df.fillna(0)

	# Substitute words in the translation based on alignments
	intermediate_list = list(zip(df["translation-tokenized"], df["proper_nouns"], df["alignments"]))

	new_translations = []
	substituted_all_info = []
	substituted_only = []
	substituted_words = []

	# Add information whether an error occurred
	error_list = []

	for i in intermediate_list:
		current_substituted_list = []
		current_substituted_only = []
		current_substituted_words = {}
		current_error = "No"

		# If no proper names were detected, do not change the translation
		if i[1] == 0:
			new_translations.append(i[0])
		
		else:
			current_translation = i[0]

			# Substitute the word with the source lemma based on the index - loop through the proper nouns to be changed
			for word_index in list(i[1].keys()):
				try:
					# split the translation into list of words
					word_list = current_translation.split()

					# Get index of the substituted word
					substituted_word_index = i[2][word_index]

					# Get the lemma to substitute the word with
					correct_lemma = i[1][word_index][1]

					# If the substitute word and lemma are not the same, get substituted word and its match
					if word_list[substituted_word_index] != correct_lemma:
						current_substituted_list.append((word_list[substituted_word_index], correct_lemma))
						current_substituted_only.append((word_list[substituted_word_index], correct_lemma))

						# Save information on which word was substituted with its conllu index (index + 1) as the key
						current_substituted_words[int(substituted_word_index+1)] = word_list[substituted_word_index]

						# Substitute the word in the word list
						word_list[substituted_word_index] = correct_lemma
					
					else:
						# Add information that substitution was not performed
						current_substituted_list.append(f"No substitution: {word_list[substituted_word_index], correct_lemma}")
					
					# Change the translation by merging the words back into a string
					current_translation = " ".join(word_list)

				except:
					print(f"Issue: index {word_index}: {i[1][word_index]}")
					current_error = f"Issue: index {word_index}: {i[1][word_index]}"

			# After the loop through proper nouns, save the new translation
			new_translations.append(current_translation)
		
		# Add information on what was substituted
		if len(substituted_all_info) != 0:
			substituted_all_info.append(current_substituted_list)
		else:
			substituted_all_info.append(0)

		if len(current_substituted_only) != 0:
			substituted_only.append(current_substituted_only)
		else:
			substituted_only.append(0)

		error_list.append(current_error)

		substituted_words.append(current_substituted_words)


	# Add to the df
	df["new_translations"] = new_translations
	df["substitution_info"] = substituted_all_info
	df["substituted_pairs"] = substituted_only
	df["substituted_words"] = substituted_words
	df["errors"] = error_list

	end_time = round((time.time() - start_time)/60,2)

	print("Alignment completed. It took {} minutes.".format(end_time))

	# Change the working directory once again
	os.chdir("..")

	# Add the word list with indices to the df
	tokenized_text_list = df.tokenized_text.to_list()
	tokenized_text_list = [i.split(" ") for i in tokenized_text_list]
	tokenized_text_dict_list = []

	for sentence in tokenized_text_list:
		sentence_list = []
		counter = 1
		for word in sentence:
			sentence_list.append([word, counter])
			counter += 1
		tokenized_text_dict_list.append(sentence_list)

	df["source_indices"] = tokenized_text_dict_list

	# Save the df
	df.to_csv("{}".format(final_dataframe), sep="\t")

	# Display most common substitutions
	df_substituted = df[df["proper_nouns"] != "0"]
	print(df_substituted.substituted_pairs.value_counts()[:20].to_markdown())
	print("\n\n")

	return df

#df = tokenize_translation(translated_dataframe_path, #translated_tokenized_dataframe_path)

#print(df.head(3).to_markdown())
#print("\n\n")

df = correct_proper_nouns(translated_tokenized_dataframe_path, final_dataframe, lang_code)

# See if there were any errors in word substitution
print("Number of errors:")
print(df[df["errors"]!="No"].shape)
print("\n\n")

# See example of sentences with substituted words
print("Example of sentences with substituted words.")
print(df[df["substituted_pairs"]!= 0][:2].to_markdown())




Alignment started.

Number of aligned sentences: 1804657


Issue: index 1: ['.', '.']
Issue: index 8: ['.', '.']
Issue: index 15: ['.', '.']
Issue: index 50: ['.', '.']
Issue: index 76: ['.', '.']
Issue: index 35: ['Zaorálek', 'Zaorálek']
Issue: index 24: ['.', '.']
Issue: index 29: ['.', '.']
Issue: index 29: ['-', '-']
Issue: index 22: ['Faltýnka', 'Faltýnek']
Issue: index 39: ['.', '.']
Issue: index 21: ['Karel', 'Karel']
Issue: index 18: ['.', '.']
Issue: index 20: ['.', '.']
Issue: index 6: ['-', '-']
Issue: index 6: ['Pane', 'pan']
Issue: index 15: ['po', 'po']
Issue: index 7: ['Bakalovo', 'Bakalův']
Issue: index 22: ['Bělobrádka', 'Bělobrádek']
Issue: index 16: ['.', '.']
Issue: index 8: ['.', '.']
Issue: index 4: ['.', '.']
Issue: index 21: ['.', '.']
Issue: index 8: ['.', '.']
Issue: index 180: ['Volný', 'Volný']
Issue: index 5: ['Kováčik', 'Kováčik']
Issue: index 201: ['Volný', 'Volný']
Issue: index 26: ['.', '.']
Issue: index 28: ['.', '.']
Issue: index 1: ['.', '.']
Issu

# Debugging space error in conllu creation

In [15]:
import pandas as pd
from stanza.utils.conll import CoNLL
import stanza
from conllu import parse
import ast
import regex as re
import os
import stanza
import time
from stanza.pipeline.core import DownloadMethod

In [16]:
# Define the language code, used in the file names
#lang_code = "CZ"
lang_code = "SI"

# Main path
main_path = "/home/tajak/Parlamint-translation"

# --------------------NO CHANGING OF THE CODE NEEDED FROM NOW ONWARDS------------------
final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

In [17]:
# Open the df
df = pd.read_csv(final_dataframe, sep="\t")
df.head(2)

,Unnamed: 0,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,translation,translation-tokenized,...,fwd_align_dict,bwd_align_dict,alignments,new_translations,substitution_info,substituted_pairs,substituted_words,errors,source_indices,post-processed_translations
0,0,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-SI_2013-09-24-SDZ6-Redna-17.conllu,ParlaMint-SI_2013-09-24-SDZ6-Redna-17.ana.seg1.1,Spoštovane kolegice poslanke in kolegi poslanc...,Spoštovane kolegice poslanke in kolegi poslanc...,{},9,"Dear fellow Members and fellow Members, ladies...","Dear fellow Members and fellow Members , ladie...",...,"{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6...","{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","Dear fellow Members and fellow Members , ladie...",0,0,{},No,"[['Spoštovane', 1], ['kolegice', 2], ['poslank...","Dear fellow Members and fellow Members , ladie..."
1,1,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-SI_2013-09-24-SDZ6-Redna-17.conllu,ParlaMint-SI_2013-09-24-SDZ6-Redna-17.ana.seg2.1,Začenjam z nadaljevanjem 17. seje Državnega zb...,Začenjam z nadaljevanjem 17. seje Državnega zb...,{},7,I begin with the continuation of the 17th sess...,I begin with the continuation of the 17th sess...,...,"{1: '1', 2: '1', 3: '2', 5: '3', 7: '3', 8: '4...","{2: '1', 3: '2', 5: '3', 8: '4', 9: '5', 12: '...","{0: 1, 1: 2, 2: 4, 3: 7, 4: 8, 5: 11, 6: 12, 7...",I begin with the continuation of the 17th sess...,[],0,{},No,"[['Začenjam', 1], ['z', 2], ['nadaljevanjem', ...",I begin with the continuation of the 17th sess...


In [18]:
# Filter out to only include the sentences that produced errors
errors = ["ParlaMint-SI_2010-12-02-SDZ5-Izredna-34.ana.seg542.1", "ParlaMint-SI_2010-03-04-SDZ5-Redna-14.ana.seg746.2", "ParlaMint-SI_2010-09-23-SDZ5-Redna-20.ana.seg112.2", "ParlaMint-SI_2010-01-29-SDZ5-Redna-13.ana.seg114.3", "ParlaMint-SI_2008-04-24-SDZ4-Redna-38.ana.seg107.4", "ParlaMint-SI_2009-10-19-SDZ5-Redna-10.ana.seg520.2", "ParlaMint-SI_2015-03-25-SDZ7-Redna-06.ana.seg350.2"]

df = df[df["sentence_id"].isin(errors)]

In [19]:
# When we open the dataframe file, the lists and dictionaries turn into strings - change them back
for column in ["space-after-information", 'fwd_align_dict', 'bwd_align_dict', 'substituted_words', "source_indices"]:
	df[column] = df[column].astype("str")
	df[column] = df[column].apply(lambda x: ast.literal_eval(x))

df.head(2)

,Unnamed: 0,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,translation,translation-tokenized,...,fwd_align_dict,bwd_align_dict,alignments,new_translations,substitution_info,substituted_pairs,substituted_words,errors,source_indices,post-processed_translations
430496,430496,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-SI_2010-12-02-SDZ5-Izredna-34.conllu,ParlaMint-SI_2010-12-02-SDZ5-Izredna-34.ana.se...,"V razumevanju vseh teh okoliščin, na podlagi p...","V razumevanju vseh teh okoliščin , na podlagi ...",{},148,"In understanding all these circumstances, on t...","In understanding all these circumstances , on ...",...,"{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6...","{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","In understanding all these circumstances , on ...",[],0,{},No,"[[V, 1], [razumevanju, 2], [vseh, 3], [teh, 4]...","In understanding all these circumstances , on ..."
489231,489231,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-SI_2010-03-04-SDZ5-Redna-14.conllu,ParlaMint-SI_2010-03-04-SDZ5-Redna-14.ana.seg7...,Na podlagi drugega odstavka 4. člena Poslovnik...,Na podlagi drugega odstavka 4. člena Poslovnik...,{},239,Pursuant to Article 4(2) of the Rules of Proce...,Pursuant to Article 4 ( 2 ) of the Rules of Pr...,...,"{1: '2', 2: '3', 3: '4', 4: '5', 5: '4', 6: '4...","{1: '1', 3: '2', 6: '3', 7: '4', 4: '5', 5: '6...","{0: 0, 1: 2, 2: 5, 3: 6, 4: 3, 5: 4, 6: 9, 7: ...",Pursuant to Article 4 ( 2 ) of the Rules of Pr...,[],0,{},No,"[[Na, 1], [podlagi, 2], [drugega, 3], [odstavk...",Pursuant to Article 4 ( 2 ) of the Rules of Pr...


In [20]:
# Create lists of information that we need to add to the conllu file
ids_list = df.sentence_id.to_list()
source_text = df.text.to_list()
# initial_translation = df.translation.to_list()
space_after_list = df["space-after-information"].to_list()
fwd_align_list = df['fwd_align_dict'].to_list()
bwd_align_list = df['bwd_align_dict'].to_list()
substituted_words_list = df['substituted_words'].to_list()
# tokenized_text_list = df["source_indices"].to_list()
sentence_list = df.new_translations.to_list()

In [22]:
# See lengths of space_after_list
for i in [space_after_list]:
    for x in i:
    	print(len(x))

203
491
508
172
250
425
509


In [23]:
nlp_Stanza = stanza.Pipeline(lang='en', processors='tokenize', tokenize_no_ssplit = True, download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True)

2023-03-10 12:41:56 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-03-10 12:41:56 INFO: Use device: gpu
2023-03-10 12:41:56 INFO: Loading: tokenize
2023-03-10 12:41:56 INFO: Done loading processors!


In [26]:
En_sentences = df.translation.to_list()

tokenized_sentences_string = []
tokenized_sentences_split = []
space_after_list = []

for i in En_sentences:
	doc = nlp_Stanza(i).to_dict()
	current_sentence_list = []
	current_space_after_list = []

	# Define a list of start_char and end_char
	start_chars = []
	end_chars = []

	# Loop through the tokens in the sentence and add them to a current sentence list
	for sentence in doc:
		for word in sentence:
			current_sentence_list.append(word["text"])

			# Add information on start and end chars to the list
			start_chars.append(word["start_char"])
			end_chars.append(word["end_char"])
		
	# Now loop through the start_char and end_char lists and find instances
	# where the end_char of one word is the same as the start_char of the next one
	# this means there is no space between them
	for char_index in range(len(start_chars)-1):
		if end_chars[char_index] == start_chars[(char_index+1)]:
			current_space_after_list.append("No")
		else:
			current_space_after_list.append("Yes")

	# This loop is not possible for the end token, so let's add information for the last token
	# just to avoid errors due to different lengths of lists
	current_space_after_list.append("Last")

	print("Length of current sentence:")
	print(len(current_sentence_list))
	#print(current_sentence_list)

	# Join the list into a space-separated string
	current_string = " ".join(current_sentence_list)
	#print(current_string)

	print("Length of the tokenized list, joined into a string and split into a list again:")
	print(len(current_string.split(" ")))

	tokenized_sentences_string.append(current_string)

	space_after_list.append(current_space_after_list)

	print("Length of SpaceAfter list:")
	print(len(current_space_after_list))

	tokenized_sentences_split.append(current_sentence_list)


Length of current sentence:
203
Length of the tokenized list, joined into a string and split into a list again:
204
Length of SpaceAfter list:
203
Length of current sentence:
491
Length of the tokenized list, joined into a string and split into a list again:
492
Length of SpaceAfter list:
491
Length of current sentence:
508
Length of the tokenized list, joined into a string and split into a list again:
509
Length of SpaceAfter list:
508
Length of current sentence:
172
Length of the tokenized list, joined into a string and split into a list again:
173
Length of SpaceAfter list:
172
Length of current sentence:
250
Length of the tokenized list, joined into a string and split into a list again:
251
Length of SpaceAfter list:
250
Length of current sentence:
425
Length of the tokenized list, joined into a string and split into a list again:
426
Length of SpaceAfter list:
425
Length of current sentence:
509
Length of the tokenized list, joined into a string and split into a list again:
510
Le

As we can see, the SpaceAfter list is of the same length as the list of tokens, but once we merge the list in a string and split the string based on spaces again, its length changes. Let's compare the two lists.

In [30]:
# Compare tokenized sentences and sentences that were created by splitting the translation sentence

sentence_list = [x.split(" ") for x in sentence_list]

for sentence_pair in list(zip(tokenized_sentences_split, sentence_list)):
    for pair in list(zip(sentence_pair[0], sentence_pair[1])):
        if pair[0] != pair[1]:
            print(pair)
            print(list(zip(sentence_pair[0], sentence_pair[1])).index(pair))
            break

('also solidarity', 'also')
184
('a public', 'a')
449
('Court of', 'Court')
223
('internal transport', 'internal')
151
('who are', 'who')
230
('European Union', 'European')
379
('Court of', 'Court')
225


In [52]:
# Try this again, but using the "spacy" pipeline

En_sentences = df.translation.to_list()

tokenized_sentences_string = []
tokenized_sentences_split = []
space_after_list = []

for i in En_sentences:
	doc = tokenizer(i).to_dict()
	current_sentence_list = []
	current_space_after_list = []

	# Define a list of start_char and end_char
	start_chars = []
	end_chars = []

	# Loop through the tokens in the sentence and add them to a current sentence list
	for sentence in doc:
		for word in sentence:
			current_sentence_list.append(word["text"])

			# Add information on start and end chars to the list
			start_chars.append(word["start_char"])
			end_chars.append(word["end_char"])
		
	# Now loop through the start_char and end_char lists and find instances
	# where the end_char of one word is the same as the start_char of the next one
	# this means there is no space between them
	for char_index in range(len(start_chars)-1):
		if end_chars[char_index] == start_chars[(char_index+1)]:
			current_space_after_list.append("No")
		else:
			current_space_after_list.append("Yes")

	# This loop is not possible for the end token, so let's add information for the last token
	# just to avoid errors due to different lengths of lists
	current_space_after_list.append("Last")

	print(len(current_sentence_list))
	#print(current_sentence_list)

	# Join the list into a space-separated string
	current_string = " ".join(current_sentence_list)
	#print(current_string)

	print(len(current_string.split(" ")))

	tokenized_sentences_string.append(current_string)

	space_after_list.append(current_space_after_list)

	#print(len(current_sentence_list))
	#print(len(current_space_after_list))

	tokenized_sentences_split.append(current_sentence_list)

204
204
488
488
509
509
173
173
251
251
426
426
510
510


In [ ]:
tokenizer_Stanza = stanza.Pipeline(lang='en', processors={'tokenize':'spacy'}, tokenize_no_ssplit = True, download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True)

In [54]:
# Define the pipeline, instruct it to use a specific package: 	CoNLL03
nlp = stanza.Pipeline(lang='en', processors="tokenize,mwt,pos,lemma,ner", package={"ner": ["conll03"]}, tokenize_pretokenized=True, download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True)

2023-03-10 11:15:05 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-03-10 11:15:06 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| ner       | conll03  |

2023-03-10 11:15:06 INFO: Use device: gpu
2023-03-10 11:15:06 INFO: Loading: tokenize
2023-03-10 11:15:06 INFO: Loading: pos
2023-03-10 11:15:06 INFO: Loading: lemma
2023-03-10 11:15:06 INFO: Loading: ner
2023-03-10 11:15:07 INFO: Done loading processors!


In [21]:
# To feed the entire list into the pipeline, we need to create lists of tokens, split by space
sentence_list = [x.split(" ") for x in sentence_list]

In [38]:
# Save the conllu file
CoNLL.write_doc2conll(doc, "{}/results/{}/temp/{}".format(main_path, lang_code, "test-space-after-debugging-tokenizer"))

In [39]:
data = open("{}/results/{}/temp/{}".format(main_path, lang_code, "test-space-after-debugging-tokenizer"), "r").read()

sentences = parse(data)

In [41]:
for sentence in sentences:
    print(sentence.serialize())

# text = In understanding all these circumstances, on the basis of the information I have been able to obtain, to think through, and to correct, even in the debates to complement both the professional services and the professional public, both the lazi public in the Slovenian space, I will somehow assess that the measure in this direction, as it was recorded before the adoption of this amendment, is a measure that balances the principle of fairness among generations so that, in the other solutions that this law brings, we can balance these burdens among all generations, provide the motive that the younger ones remain in the solidarity system, that we are willing to give a certain value to contributions to pension commodities, so that, also on the basis of this same solidarity, we will be assured to survive a decent age in the future and, of course, the motive for young people entering the labour relationship, that it is reasonable to maintain this system of intergenerational solidarity

In [28]:
for sentence in sentences:
    print(sentence.metadata["text"])

In understanding all these circumstances , on the basis of the information I have been able to obtain , to think through , and to correct , even in the debates to complement both the professional services and the professional public , both the lazi public in the Slovenian space , I will somehow assess that the measure in this direction , as it was recorded before the adoption of this amendment , is a measure that balances the principle of fairness among generations so that , in the other solutions that this law brings , we can balance these burdens among all generations , provide the motive that the younger ones remain in the solidarity system , that we are willing to give a certain value to contributions to pension commodities , so that , also on the basis of this same solidarity , we will be assured to survive a decent age in the future and , of course , the motive for young people entering the labour relationship , that it is reasonable to maintain this system of intergenerational s

In [55]:
# Adding additional information to the conllu
for sentence in sentences:
	# Get the sentence index
	sentence_index = sentences.index(sentence)

	# Add metadata
	sentence.metadata["sent_id"] = ids_list[sentence_index]
	sentence.metadata["source"] = source_text[sentence_index]
	# sentence.metadata["source_indices"] = tokenized_text_list[sentence_index]
	# sentence.metadata["initial_translation"] = initial_translation[sentence_index]

	# Delete the current metadata for text
	del sentence.metadata["text"]

	new_translation_text = ""

	# Iterate through tokens
	for word in sentence:
		word_index = sentence.index(word)
		word_conllu_index = word["id"]

		# Check whether the word conllu index (word id) is in the substituted_words_list (it is if it was substituted)
		# If it is, add information on the original translated word - do not do this for Bulgarian and Portuguese
		if lang_code not in ["BG", "PT", "IT", "AT", "GR", "HU", "NO", "TR", "NL", "SI"]:
			if substituted_words_list[sentence_index].get(word_conllu_index, None) != None:
				word["misc"]["Translated"] = substituted_words_list[sentence_index][word_conllu_index]
		
		# Do the same for the forward and backward alignment
		if fwd_align_list[sentence_index].get(word_conllu_index, None) != None:
			word["misc"]["ForwardAlignment"] = fwd_align_list[sentence_index][word_conllu_index]

		if bwd_align_list[sentence_index].get(word_conllu_index, None) != None:
			word["misc"]["BackwardAlignment"] = bwd_align_list[sentence_index][word_conllu_index]

		# Remove information on start_char and end_char from the annotation
		del word["misc"]["start_char"]
		del word["misc"]["end_char"]
		
		# Change the NER tags so that they are the same as in the source
		current_ner = word["misc"]["ner"]
		del word["misc"]["ner"]
		
		# Substitute parts of the tags so that they are the same as in source
		current_ner = re.sub("S-", "B-", current_ner)
		current_ner = re.sub("E-", "I-", current_ner)

		word["misc"]["NER"] = current_ner

		try:
			# Get information about the space after based on the index
			current_space_after = space_after_list[sentence_index][word_index]
		except:
			print("Error based on current_space after in sentence {}, sentence index: {}, word {}, word index {}.".format(sentence, sentence_index, word, word_index))
			current_space_after = "Yes"

	# Create new text from translation, correcting the spaces around words
	# based on the SpaceAfter information
		if current_space_after == "No":
			word["misc"]["SpaceAfter"] = "No"
			new_translation_text += word["form"]
		elif current_space_after == "Last":
			new_translation_text += word["form"]
		else:
			new_translation_text += word["form"]
			new_translation_text += " "
	
	sentence.metadata["text"] = new_translation_text

# Create a new conllu file with the updated information
final_file = open("{}/results/{}/temp/{}".format(main_path, lang_code, "test-space-after-debugging-final-conllu"), "w")

for sentence in sentences:
	final_file.write(sentence.serialize())

final_file.close()

KeyError: 'text'

## Debugging alignment in new DK

In [2]:
# Define the language code, used in the file names
#lang_code = "CZ"
lang_code = "DK"

# Main path
main_path = "/home/tajak/Parlamint-translation"

# --------------------NO CHANGING OF THE CODE NEEDED FROM NOW ONWARDS------------------
import pandas as pd

translated_dataframe_path = "{}/results/{}/ParlaMint-{}-translated.csv".format(main_path, lang_code, lang_code)
translated_tokenized_dataframe_path = "{}/results/{}/ParlaMint-{}-translated-tokenized.csv".format(main_path,lang_code, lang_code)
final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

from knockknock import discord_sender
# Create a dictionary from the returned alignment files which will be added to each word in the final conllu

def alignment_file_to_target_dict(file):
	"""
	The output of the eflomal aligner is in the source to target direction. We want to get the alignments in the other direction
	and for each target word add to the conllu its aligned source word index (as it appears in conllu). In conllu, indices start
	with 1, not 0. So, we take the eflomal files, reverse the order and create dictionaries with target indexes as keys
	and source indexes as values. If there are more than one words aligned to the same target word, it looks like this: '1, 2'.
	We use the conllu indexes which means that we add 1 to each index in the alingment pairs. 

	Args:
		- file: the path to the .fwd and .rev file that is produced by the eflomal tool

	The result is a list of dictionaries, each dictionary corresponds to one sentence.
	"""
	# Create target alignments from the source alignment direction (by changing the direction in the file)
	aligns_list_target = open(file, "r").readlines()
	aligns_list_target = [i.replace("\n", "") for i in aligns_list_target]
	aligns_list_target = [i.split(" ") for i in aligns_list_target]

	aligns_list_target_dict_list = []

	# Loop through the alignments for sentences
	for i in aligns_list_target:
		# Create a dictionary for each sentence
		current_sentence_align = {}
		# If alignment line is empty, keep the dictionary empty
		if len(i) == 1 and len(i[0]) == 0:
			current_sentence_align = {}
		else:
			# For each alignment pair in the sentence:
			for pair in i:
				# Split the pair: result is a list of lists with source index as the first element
				# and target index as the second element: [[0,0], [1,2], [1,3]]
				current_pair = pair.split("-")

				# Get the indices for target and source and add 1 to them (to get the conllu indices)
				current_t_index = int(current_pair[1]) + 1
				current_s_index = int(current_pair[0]) + 1

				# Check whether the target index is already aligned to anything (a case of 1-to-many alignment),
				# if not, save it as a key and save the source index as value.
				if current_sentence_align.get(current_t_index, None) == None:
					current_sentence_align[current_t_index] = str(current_s_index)
				# If the index was already aligned to a previous source word, add the additional source word alignment as a string
				# (result: {0: "1, 2"))
				else:
					current_sentence_align[current_t_index] += str(", ")
					current_sentence_align[current_t_index] += str(current_s_index)

		aligns_list_target_dict_list.append(current_sentence_align)

	return aligns_list_target_dict_list

def correct_proper_nouns(translated_tokenized_dataframe_path, final_dataframe, lang_code):
	"""
	This function takes the translated text and the source text, aligns words with eflomal and corrects proper nouns.
	It takes the dataframe that was created in the function extract_text() and to which the translation was added
	in the function translate().

	To use eflomal, you need to install it first:
	!git clone https://github.com/robertostling/eflomal
	%cd eflomal
	!make
	!sudo make install
	!python3 setup.py install

	In case you don't have sudo permission, you can skip !sudo make install. I did, and I also used a virtual environment (venv), and managed to install eflomal.

	"""
	import pandas as pd
	import re
	import ast
	from IPython.display import display
	import time
	import subprocess
	import os

	# Open the file, created in the previous step
	df = pd.read_csv("{}".format(translated_tokenized_dataframe_path), sep="\t", index_col=0, na_filter = False)

	# Move into the eflomal folder
	os.chdir("/home/tajak/Parlamint-translation/eflomal")

	# Then we need to create files for all texts and all translations
	source_sentences = open("source_sentences_{}.txt".format(lang_code), "w")
	English_sentences = open("English_sentences_{}.txt".format(lang_code), "w")

	for i in df["tokenized_text"].to_list():
		source_sentences.write(i)
		source_sentences.write("\n")

	for i in df["translation-tokenized"].to_list():
		English_sentences.write(i)
		English_sentences.write("\n")

	source_sentences.close()
	English_sentences.close()

	print("\n\n")
	print("Alignment started.")
	start_time = time.time()

	# Align sentences with eflomal and get out a file with alignments
	#!python3 align.py -s source_sentences.txt -t English_sentences.txt --model 3 -r source-en.rev -f source-en.fwd
	subprocess.call(["/home/tajak/Parlamint-translation/align.sh", lang_code])

	# Create a list of dictionaries of alignments from the returned files which will be added to the final conllu for each word
	forward_alignment_dict_list = alignment_file_to_target_dict("source-en_{}.fwd".format(lang_code))
	backward_alignment_dict_list = alignment_file_to_target_dict("source-en_{}.rev".format(lang_code))

	# Add to the df
	df["fwd_align_dict"] = forward_alignment_dict_list
	df["bwd_align_dict"] = backward_alignment_dict_list

	# Create forward target alignments from the source alignment direction (by changing the direction in the rev file)
	aligns_list = open("source-en_{}.rev".format(lang_code), "r").readlines()
	aligns_list = [i.replace("\n", "") for i in aligns_list]

	# Continue with processing the list to create the final alignments format which I'll use to correct proper names
	aligns_list = [i.split(" ") for i in aligns_list]

	for i in aligns_list:
		# If alignment line is empty, keep the dictionary empty
		if len(i) == 1 and len(i[0]) == 0:
			aligns_list[aligns_list.index(i)] = []
		else:
			for pair in i:
				current_pair = pair.split("-")
				i[i.index(pair)] = {int(current_pair[0]): int(current_pair[1])}
	
	final_aligns = []

	# Create a dictionary out of the rev alignments
	for i in aligns_list:
		current_line = {}

		try:
			for element in i:
				a = list(element.items())[0][0]
				b = list(element.items())[0][1]
				current_line[a] = b
		
			# Check whether the number of pairs in the list is the same as number of items
			if len(i) != len(list(current_line.items())):
				print("Not okay:")
				print(i)
				print(current_line)

			final_aligns.append(current_line)
		
		except:
			print("error")
			print(aligns_list.index(i))
			print(i)
			final_aligns.append("Error")
		
	print("\nNumber of aligned sentences: {}\n\n".format(len(final_aligns)))

	# Add a to the df
	df["alignments"] = final_aligns

	# When we open the dataframe file, the dictionaries with proper names changed into strings - Change strings in the column proper_nouns into dictionaries

	df["proper_nouns"] = df.proper_nouns.astype("str")
	df["proper_nouns"] = df.proper_nouns.apply(lambda x: ast.literal_eval(x))

	# Change nan values in the proper_nouns columns
	df = df.fillna(0)

	# Substitute words in the translation based on alignments
	intermediate_list = list(zip(df["translation-tokenized"], df["proper_nouns"], df["alignments"]))

	new_translations = []
	substituted_all_info = []
	substituted_only = []
	substituted_words = []

	# Add information whether an error occurred
	error_list = []

	for i in intermediate_list:
		current_substituted_list = []
		current_substituted_only = []
		current_substituted_words = {}
		current_error = "No"

		# If no proper names were detected, do not change the translation
		if i[1] == 0:
			new_translations.append(i[0])
		
		else:
			current_translation = i[0]

			# Substitute the word with the source lemma based on the index - loop through the proper nouns to be changed
			for word_index in list(i[1].keys()):
				try:
					# split the translation into list of words
					word_list = current_translation.split()

					# Get index of the substituted word
					substituted_word_index = i[2][word_index]

					# Get the lemma to substitute the word with
					correct_lemma = i[1][word_index][1]

					# If the substitute word and lemma are not the same, get substituted word and its match
					if word_list[substituted_word_index] != correct_lemma:
						current_substituted_list.append((word_list[substituted_word_index], correct_lemma))
						current_substituted_only.append((word_list[substituted_word_index], correct_lemma))

						# Save information on which word was substituted with its conllu index (index + 1) as the key
						current_substituted_words[int(substituted_word_index+1)] = word_list[substituted_word_index]

						# Substitute the word in the word list
						word_list[substituted_word_index] = correct_lemma
					
					else:
						# Add information that substitution was not performed
						current_substituted_list.append(f"No substitution: {word_list[substituted_word_index], correct_lemma}")
					
					# Change the translation by merging the words back into a string
					current_translation = " ".join(word_list)

				except:
					print(f"Issue: index {word_index}: {i[1][word_index]}")
					current_error = f"Issue: index {word_index}: {i[1][word_index]}"

			# After the loop through proper nouns, save the new translation
			new_translations.append(current_translation)
		
		# Add information on what was substituted
		if len(substituted_all_info) != 0:
			substituted_all_info.append(current_substituted_list)
		else:
			substituted_all_info.append(0)

		if len(current_substituted_only) != 0:
			substituted_only.append(current_substituted_only)
		else:
			substituted_only.append(0)

		error_list.append(current_error)

		substituted_words.append(current_substituted_words)


	# Add to the df
	df["new_translations"] = new_translations
	df["substitution_info"] = substituted_all_info
	df["substituted_pairs"] = substituted_only
	df["substituted_words"] = substituted_words
	df["errors"] = error_list

	end_time = round((time.time() - start_time)/60,2)

	print("Alignment completed. It took {} minutes.".format(end_time))

	# Change the working directory once again
	os.chdir("..")

	# Add the word list with indices to the df
	tokenized_text_list = df.tokenized_text.to_list()
	tokenized_text_list = [i.split(" ") for i in tokenized_text_list]
	tokenized_text_dict_list = []

	for sentence in tokenized_text_list:
		sentence_list = []
		counter = 1
		for word in sentence:
			sentence_list.append([word, counter])
			counter += 1
		tokenized_text_dict_list.append(sentence_list)

	df["source_indices"] = tokenized_text_dict_list

	# Save the df
	df.to_csv("{}".format(final_dataframe), sep="\t")

	# Display most common substitutions
	df_substituted = df[df["proper_nouns"] != "0"]
	print(df_substituted.substituted_pairs.value_counts()[:20].to_markdown())
	print("\n\n")

	return df

In [6]:
df = correct_proper_nouns(translated_tokenized_dataframe_path, final_dataframe, lang_code)

# See if there were any errors in word substitution
print("Number of errors:")
print(df[df["errors"]!="No"].shape)
print("\n\n")

# See example of sentences with substituted words
print("Example of sentences with substituted words.")
print(df[df["substituted_pairs"]!= 0][:2].to_markdown())




Alignment started.

Number of aligned sentences: 1978777


Issue: index 8: ['Hav', 'Hav']
Issue: index 561: ['Jensen', 'Jensen']
Issue: index 578: ['Tørnæs', 'Tørnæs']
Issue: index 2: ['Mogens', 'Mogens']
Issue: index 3: ['Lykketoft', 'Lykketoft']
Issue: index 9: ['Bertel', 'Bertel']
Issue: index 10: ['Haarder', 'Haarder']
Issue: index 28: ['Julie', 'Julie']
Issue: index 29: ['Skovsby', 'Skovsby']
Issue: index 34: ['Karen', 'Karen']
Issue: index 35: ['J.', 'J.']
Issue: index 36: ['Klint', 'Klint']
Issue: index 41: ['Ane', 'Ane']
Issue: index 42: ['Halsboe-Jørgensen', 'Halsboe-Jørgensen']
Issue: index 47: ['Mette', 'Mette']
Issue: index 48: ['Gjerskov', 'Gjerskov']
Issue: index 53: ['Thomas', 'Thomas']
Issue: index 54: ['Jensen', 'Jensen']
Issue: index 59: ['Camilla', 'Camilla']
Issue: index 60: ['Hersom', 'Hersom']
Issue: index 65: ['Karina', 'Karina']
Issue: index 66: ['Lorentzen', 'Lorentzen']
Issue: index 67: ['Dehnhardt', 'Dehnhardt']
Issue: index 72: ['Frank', 'Frank']
Issue: i